In [1]:
import os
import time
import datetime
import numpy as np
import pandas as pd
import sklearn
import torch

FILE_NAME_SUFFIX = ".multi_admission" #"_1000"
DATASET_PATH = "Datasets/"

def read_samples(input_file, file_name_suffix):
    file_name = DATASET_PATH + input_file + file_name_suffix
    return pd.read_csv(file_name, error_bad_lines=False, keep_default_na=False)


In [2]:
patients = read_samples("PATIENTS.csv", FILE_NAME_SUFFIX)
patients['birth_date'] = patients["DOB"].apply(lambda s: (datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S') - datetime.datetime(1970,1,1)).days)

admissions = read_samples("ADMISSIONS.csv", FILE_NAME_SUFFIX)
admissions["admit_date"] = admissions["ADMITTIME"].apply(lambda s: (datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S') - datetime.datetime(1970,1,1)).days)
admissions["discharge_date"] = admissions["DISCHTIME"].apply(lambda s: (datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S') - datetime.datetime(1970,1,1)).days)
admissions['LOS'] = admissions["discharge_date"] - admissions["admit_date"]

diagnoses = read_samples("DIAGNOSES_ICD.csv", FILE_NAME_SUFFIX)
procedures = read_samples("PROCEDURES_ICD.csv", FILE_NAME_SUFFIX)
icu_stays = read_samples("ICUSTAYS.csv", FILE_NAME_SUFFIX)
notes = read_samples("NOTEEVENTS.csv", FILE_NAME_SUFFIX)


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
patients = patients.set_index("SUBJECT_ID", drop=False)
patients["num_admissions"] = admissions.groupby("SUBJECT_ID").size().to_frame("num_admissions")
patients = patients[patients.num_admissions > 1]
print("Limit the patients to the ones with more than 1 admission:\n", patients.num_admissions.describe())
print("Prevalence of patients with more than 1 admission: ", len(patients[patients["num_admissions"] > 1]) / len(patients))
print("Prevalence of patients with more than 2 admissions: ", len(patients[patients["num_admissions"] > 2]) / len(patients))
print("Prevalence of patients with more than 3 admissions: ", len(patients[patients["num_admissions"] > 3]) / len(patients))
print("Prevalence of patients with more than 4 admissions: ", len(patients[patients["num_admissions"] > 4]) / len(patients))
print("Prevalence of patients with more than 5 admissions: ", len(patients[patients["num_admissions"] > 5]) / len(patients))

Limit the patients to the ones with more than 1 admission:
 count    7537.000000
mean        2.652647
std         1.621112
min         2.000000
25%         2.000000
50%         2.000000
75%         3.000000
max        42.000000
Name: num_admissions, dtype: float64
Prevalence of patients with more than 1 admission:  1.0
Prevalence of patients with more than 2 admissions:  0.31537747114236436
Prevalence of patients with more than 3 admissions:  0.13732254212551412
Prevalence of patients with more than 4 admissions:  0.06992171951704923
Prevalence of patients with more than 5 admissions:  0.03728273849011543


In [4]:
admissions = admissions[admissions.SUBJECT_ID.isin(patients.SUBJECT_ID)]
procedures = procedures[procedures.SUBJECT_ID.isin(patients.SUBJECT_ID)]
diagnoses = diagnoses[diagnoses.SUBJECT_ID.isin(patients.SUBJECT_ID)]
icu_stays = icu_stays[icu_stays.SUBJECT_ID.isin(patients.SUBJECT_ID)]
notes = notes[notes.SUBJECT_ID.isin(patients.SUBJECT_ID)]

In [5]:
last_admission = admissions[admissions.groupby(['SUBJECT_ID'])['admit_date'].transform(max) == admissions['admit_date']]
previous_admissions = admissions[admissions.groupby(['SUBJECT_ID'])['admit_date'].transform(max) != admissions['admit_date']]
patients["record_start_date"] = previous_admissions.groupby("SUBJECT_ID").admit_date.agg(['min'])
patients["record_end_date"] = previous_admissions.groupby("SUBJECT_ID").discharge_date.agg(['max'])
patients["record_length"] = patients.record_end_date - patients.record_start_date
patients["final_admission_date"] = admissions.groupby("SUBJECT_ID").admit_date.agg(['max'])
patients["final_admission_interval"] = patients.final_admission_date - patients.record_end_date

print("Record length before the final admission\n", patients.record_length.describe())

print("Interval before the final admission\n", patients.final_admission_interval.describe())

print("Prevalence of readmission in 30 days = ", len(patients[patients.final_admission_interval < 30]) / len(patients))


Record length before the final admission
 count    7533.000000
mean      242.149608
std       579.288149
min         0.000000
25%         6.000000
50%        13.000000
75%        84.000000
max      4145.000000
Name: record_length, dtype: float64
Interval before the final admission
 count    7533.000000
mean      445.034382
std       678.907514
min       -19.000000
25%        25.000000
50%       131.000000
75%       570.000000
max      4108.000000
Name: final_admission_interval, dtype: float64
Prevalence of readmission in 30 days =  0.27517579938967757


In [6]:
icu_stays["admit_date"] = icu_stays["INTIME"].apply(lambda s: (datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S') - datetime.datetime(1970,1,1)).days)
final_icu_admission = icu_stays[icu_stays.groupby(['SUBJECT_ID'])['admit_date'].transform(max) != icu_stays['admit_date']]
patients["final_icu_admission_date"] = final_icu_admission.groupby("SUBJECT_ID").admit_date.agg(['min'])
patients["final_icu_admission_interval"] = patients.final_icu_admission_date - patients.record_end_date
print("Prevalence of ICU admission in 30 days = ", len(patients[(patients["final_icu_admission_interval"] >= 0) & (patients["final_icu_admission_interval"] < 30)]) / len(patients))


Prevalence of ICU admission in 30 days =  0.006103224094467295


In [7]:
print(procedures.groupby("ICD9_CODE").size().to_frame("procedure_freq_by_icd9_code").describe())
print(procedures.groupby("SUBJECT_ID").size().to_frame("procedure_freq_by_patient").describe())
print(procedures.groupby("HADM_ID").size().to_frame("procedure_freq_by_admission").describe())

print(procedures.groupby("ICD9_CODE").size().to_frame("freq").sort_values("freq", ascending=False).head(20))

       procedure_freq_by_icd9_code
count                  1513.000000
mean                     54.035030
std                     275.114179
min                       1.000000
25%                       1.000000
50%                       4.000000
75%                      19.000000
max                    6505.000000
       procedure_freq_by_patient
count                7364.000000
mean                   11.101983
std                     8.891169
min                     1.000000
25%                     5.000000
50%                     9.000000
75%                    15.000000
max                    98.000000
       procedure_freq_by_admission
count                 17393.000000
mean                      4.700454
std                       3.961372
min                       1.000000
25%                       2.000000
50%                       3.000000
75%                       6.000000
max                      40.000000
           freq
ICD9_CODE      
3893       6505
9604       3440
966      

In [8]:
print(diagnoses.groupby("ICD9_CODE").size().to_frame("diagnosis_freq_by_icd9_code").describe())
print(diagnoses.groupby("SUBJECT_ID").size().to_frame("diagnosis_freq_by_patient").describe())
print(diagnoses.groupby("HADM_ID").size().to_frame("diagnosis_freq_by_admission").describe())

print(diagnoses.groupby("ICD9_CODE").size().to_frame("freq").sort_values("freq", ascending=False).head(20))

       diagnosis_freq_by_icd9_code
count                  4894.000000
mean                     53.192889
std                     258.042880
min                       1.000000
25%                       1.000000
50%                       4.000000
75%                      20.000000
max                    7183.000000
       diagnosis_freq_by_patient
count                7537.000000
mean                   34.539737
std                    28.731059
min                     2.000000
25%                    18.000000
50%                    28.000000
75%                    41.000000
max                   540.000000
       diagnosis_freq_by_admission
count                 19993.000000
mean                     13.020857
std                       6.860812
min                       1.000000
25%                       9.000000
50%                      11.000000
75%                      17.000000
max                      39.000000
           freq
ICD9_CODE      
4019       7183
4280       6588
42731    

In [9]:
notes['text_len'] = notes['TEXT'].apply(lambda s: len(s.split()))
print(notes["text_len"].describe())
print(notes.groupby("SUBJECT_ID").size().to_frame("notes_freq_by_patient").describe())
print(notes.groupby("HADM_ID").size().to_frame("notes_freq_by_admission").describe())

count    739127.000000
mean        280.506953
std         381.865602
min           0.000000
25%          72.000000
50%         160.000000
75%         318.000000
max        7980.000000
Name: text_len, dtype: float64
       notes_freq_by_patient
count            7535.000000
mean               98.092502
std               117.930373
min                 1.000000
25%                34.000000
50%                62.000000
75%               116.000000
max              1420.000000
       notes_freq_by_admission
count             19758.000000
mean                 37.408999
std                 948.520727
min                   1.000000
25%                   8.000000
50%                  15.000000
75%                  31.000000
max              133139.000000


In [10]:
discharge_summaries = notes[notes.CATEGORY == "Discharge summary"]
print(discharge_summaries["text_len"].describe())
print(discharge_summaries.groupby("SUBJECT_ID").size().to_frame("notes_freq_by_patient").describe())
print(discharge_summaries.groupby("HADM_ID").size().to_frame("notes_freq_by_admission").describe())

count    21740.000000
mean      1603.497148
std        884.371393
min          9.000000
25%       1008.000000
50%       1527.000000
75%       2111.000000
max       7980.000000
Name: text_len, dtype: float64
       notes_freq_by_patient
count            7451.000000
mean                2.917729
std                 1.935627
min                 1.000000
25%                 2.000000
50%                 2.000000
75%                 3.000000
max                47.000000
       notes_freq_by_admission
count             19050.000000
mean                  1.141207
std                   0.438223
min                   1.000000
25%                   1.000000
50%                   1.000000
75%                   1.000000
max                   7.000000


In [11]:
patients["death_date"] = patients["DOD"].apply(lambda s: (datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S') - datetime.datetime(1970,1,1)).days if s != '' else np.nan)
patients['death_interval'] = patients.death_date - patients.record_end_date
print(patients['death_interval'].describe())
print("Prevalence of death in 30 days = ", len(patients[patients.death_interval < 30]) / len(patients))
print("Prevalence of death = ", len(patients[patients.death_interval >= 0]) / len(patients))

count    3902.000000
mean      663.113275
std       811.485606
min         0.000000
25%        80.250000
50%       316.500000
75%       937.750000
max      4328.000000
Name: death_interval, dtype: float64
Prevalence of death in 30 days =  0.05532705320419265
Prevalence of death =  0.5177126177524214


In [12]:
# Limit the procedures to the most common procedures
NUM_PROCEDURE_CODES = 1024
top_procedures = procedures.groupby("ICD9_CODE").size().to_frame("freq").sort_values("freq", ascending=False).head(NUM_PROCEDURE_CODES).index.tolist()
procedures = procedures[procedures.ICD9_CODE.isin(top_procedures)]
print(procedures.groupby("ICD9_CODE").size().to_frame("procedure_freq_by_icd9_code").describe())
print(procedures.groupby("SUBJECT_ID").size().to_frame("procedure_freq_by_patient").describe())
print(procedures.groupby("HADM_ID").size().to_frame("procedure_freq_by_admission").describe())

       procedure_freq_by_icd9_code
count                  1024.000000
mean                     79.286133
std                     331.499928
min                       2.000000
25%                       4.000000
50%                      10.500000
75%                      38.000000
max                    6505.000000
       procedure_freq_by_patient
count                7361.000000
mean                   11.029616
std                     8.854558
min                     1.000000
25%                     5.000000
50%                     9.000000
75%                    15.000000
max                    98.000000
       procedure_freq_by_admission
count                 17355.000000
mean                      4.678133
std                       3.937743
min                       1.000000
25%                       2.000000
50%                       3.000000
75%                       6.000000
max                      39.000000


In [13]:
# Limit the diagnoses to the most common diagnoses
NUM_DIAGNOSIS_CODES = 4096
top_diagnoses = diagnoses.groupby("ICD9_CODE").size().to_frame("freq").sort_values("freq", ascending=False).head(NUM_DIAGNOSIS_CODES).index.tolist()
diagnoses = diagnoses[diagnoses.ICD9_CODE.isin(top_diagnoses)]
print(diagnoses.groupby("ICD9_CODE").size().to_frame("diagnosis_freq_by_icd9_code").describe())
print(diagnoses.groupby("SUBJECT_ID").size().to_frame("diagnosis_freq_by_patient").describe())
print(diagnoses.groupby("HADM_ID").size().to_frame("diagnosis_freq_by_admission").describe())

       diagnosis_freq_by_icd9_code
count                  4096.000000
mean                     63.361328
std                     280.940588
min                       1.000000
25%                       2.000000
50%                       7.000000
75%                      28.000000
max                    7183.000000
       diagnosis_freq_by_patient
count                7537.000000
mean                   34.433860
std                    28.696845
min                     2.000000
25%                    18.000000
50%                    27.000000
75%                    41.000000
max                   539.000000
       diagnosis_freq_by_admission
count                 19990.000000
mean                     12.982891
std                       6.851016
min                       1.000000
25%                       9.000000
50%                      11.000000
75%                      17.000000
max                      39.000000


In [14]:
from torch.utils.data import Dataset

other_admission_info_dim = 2 # age and LOS

class CustomDataset(Dataset):
    
    def __init__(self, patients, admissions, procedures, top_procedures, diagnoses, top_diagnoses, prediction_window):
        top_procedures_dict = dict(zip(top_procedures, range(len(top_procedures))))
        top_diagnoses_dict = dict(zip(top_diagnoses, range(len(top_procedures), len(top_procedures) + len(top_diagnoses))))
        self.x = []
        self.y = []
        for _, patient in patients.iterrows():
            patient_admissions = []
            for _, admission in admissions[admissions.SUBJECT_ID == patient.SUBJECT_ID].iterrows():
                icd9_codes = []
                for _, admission_procedure in procedures[procedures.HADM_ID == admission.HADM_ID].iterrows():
                    icd9_codes.append(top_procedures_dict[admission_procedure.ICD9_CODE])
                for _, admission_diagnosis in diagnoses[diagnoses.HADM_ID == admission.HADM_ID].iterrows():
                    icd9_codes.append(top_diagnoses_dict[admission_diagnosis.ICD9_CODE])
                other_info = [(admission.admit_date - patient.birth_date) / 36500.0, admission.LOS / 100.0]
                patient_admissions.append((other_info, icd9_codes))
            self.x.append(patient_admissions)
            self.y.append([patient.final_admission_interval < prediction_window, patient.death_interval < prediction_window])
            # self.y.append(patient.final_admission_interval < prediction_window)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

dataset = CustomDataset(patients, previous_admissions, procedures, top_procedures, diagnoses, top_diagnoses, 30)
print(len(dataset))
# for i in range(len(dataset)):
#     x, y = dataset[i]
#     print(y)

7537


In [15]:
def collate_fn(data):
    """
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
        sequences to the sample shape (max # visits, encoding size). The padding infomation
        is stored in `mask`.
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, encoding size) of type torch.long
        masks: a tensor of shape (# patiens, max # visits, encoding size) of type torch.bool
        y: a tensor of shape (# patiens) of type torch.float
    """

    sequences, labels = zip(*data)
    num_patients = len(sequences)
    max_admissions = 0
    max_icd9_codes = 0
    for sequence in sequences:
        max_admissions = max(max_admissions, len(sequence))
        for admission in sequence:
            (other_info, icd9_codes) = admission
            max_icd9_codes = max(max_icd9_codes, len(icd9_codes))
    
    dim = (num_patients, max_admissions, max_icd9_codes)
    x_data = np.zeros(dim)
    x_other_data = np.zeros((num_patients, max_admissions, other_admission_info_dim))
    masks_data = np.full(dim, False)
    
    for i, sequence in enumerate(sequences):
        num_admissions = len(sequence)
        for j, admission in enumerate(sequence):
            (other_info, icd9_codes) = admission
            for k, icd9_code in enumerate(icd9_codes):
                x_data[i][j][k] = icd9_code
                masks_data[i][j][k] = True
            for k, other_admission_info in enumerate(other_info):
                x_other_data[i][j][k] = other_admission_info
       
    x = torch.tensor(x_data, dtype=torch.long)
    x_other = torch.tensor(x_other_data, dtype=torch.float)
    masks = torch.tensor(masks_data, dtype=torch.bool)
    y = torch.tensor(labels, dtype=torch.float)
    
    return x, x_other, masks, y

In [16]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=10, collate_fn=collate_fn)
loader_iter = iter(loader)
x, x_other, masks, y = next(loader_iter)
print(x.shape, x_other.shape, masks.shape, y.shape)

torch.Size([10, 5, 39]) torch.Size([10, 5, 2]) torch.Size([10, 5, 39]) torch.Size([10, 2])


In [17]:
from torch.utils.data.dataset import random_split

split = int(len(dataset)*0.75)

lengths = [split, len(dataset) - split]
train_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 5652
Length of val dataset: 1885


In [18]:
from torch.utils.data import DataLoader

def load_data(train_dataset, val_dataset, collate_fn):
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
    
    return train_loader, val_loader


train_loader, val_loader = load_data(train_dataset, val_dataset, collate_fn)
combined_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [19]:
def sum_embeddings_with_mask(x, masks):
    (batch_size, visits, diags, embedding_dim) = x.shape
    masks = masks.unsqueeze(3).expand(batch_size, visits, diags, embedding_dim)
    output = torch.sum(x * masks, dim=2)
    return output

In [20]:
def get_last_visit(hidden_states, masks):
    batch_size, visits, embedding_dim = hidden_states.shape
    masks = torch.sum(masks, 2)
    masks = torch.min(masks, torch.ones_like(masks))
    masks = torch.sum(masks, 1)
    masks = masks - torch.ones_like(masks)
    masks = masks.unsqueeze(1).expand(batch_size, embedding_dim).unsqueeze(1)
    masks = torch.max(masks, torch.zeros_like(masks)) # FIXME: data cleaning problem! some patients have no admission
    last_visit = torch.gather(hidden_states, 1, masks)
    last_visit = torch.flatten(last_visit, 1, 2)
    return last_visit

In [21]:
class NaiveRNN(torch.nn.Module):
    def __init__(self, num_embeddings, embedding_size, other_admission_info_dim, hidden_state_size, output_size):
        super().__init__()
        self.hidden_state_size = hidden_state_size
        self.embedding = torch.nn.Embedding(num_embeddings, embedding_size)
        self.rnn = torch.nn.GRU(embedding_size + other_admission_info_dim, hidden_state_size, batch_first=True)
        self.linear1 = torch.nn.Linear(hidden_state_size, hidden_state_size)
        self.activation1 = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.5)
        self.linear2 = torch.nn.Linear(hidden_state_size, output_size)
        self.activation2 = torch.nn.Sigmoid()
    
    
    def forward(self, x, x_other, masks):
        (batch_size, num_admissions, _) = x.shape
        hidden_state = torch.zeros(1, batch_size, self.hidden_state_size)
        hidden_states = []
        embeddings = self.embedding(x)
        sum_embeddings = sum_embeddings_with_mask(embeddings, masks)
        combined_admission_info = torch.cat((sum_embeddings, x_other), 2)
        output, _ = self.rnn(combined_admission_info)
        output = get_last_visit(output, masks)
        output = self.activation1(self.linear1(output))
        output = self.dropout(output)
        output = self.activation2(self.linear2(output))
        return output.squeeze()
    

# load the model here
naive_rnn = NaiveRNN(num_embeddings=len(top_procedures)+len(top_diagnoses), embedding_size = 16, other_admission_info_dim=other_admission_info_dim, hidden_state_size=16, output_size=2)
naive_rnn

NaiveRNN(
  (embedding): Embedding(5120, 16)
  (rnn): GRU(18, 16, batch_first=True)
  (linear1): Linear(in_features=16, out_features=16, bias=True)
  (activation1): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (linear2): Linear(in_features=16, out_features=2, bias=True)
  (activation2): Sigmoid()
)

In [22]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(naive_rnn.parameters(), lr=0.001)

In [28]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score


def eval_model(model, val_loader):
    model.eval()
    Y1_pred = []
    Y1_true = []
    Y2_pred = []
    Y2_true = []
    for x, x_other, masks, y in val_loader:
        with torch.no_grad():
            pred = model(x, x_other, masks)
            # Y1_true.extend(y.detach().numpy().tolist())
            # Y1_pred.extend(pred.detach().numpy().reshape(-1).tolist())
            Y1_true.extend(y.detach().numpy()[:,0].tolist())
            Y1_pred.extend(pred.detach().numpy()[:,0].reshape(-1).tolist())
            Y2_true.extend(y.detach().numpy()[:,1].tolist())
            Y2_pred.extend(pred.detach().numpy()[:,1].reshape(-1).tolist())
    
    # print(len(Y1_true), len(Y1_pred), len(Y2_true), len(Y2_pred))
    precision1, recall1, f11, _ = precision_recall_fscore_support(Y1_true, np.array(Y1_pred)>0.5, average='binary')
    roc_auc1 = roc_auc_score(Y1_true, Y1_pred)
    precision2, recall2, f12, _ = precision_recall_fscore_support(Y2_true, np.array(Y2_pred)>0.5, average='binary')
    roc_auc2 = roc_auc_score(Y2_true, Y2_pred)
    

    
    return precision1, recall1, f11, roc_auc1, precision2, recall2, f12, roc_auc2, len(Y1_pred), sum(Y1_true), sum(Y2_true)

In [29]:
# precision1, recall1, f11, roc_auc1 = eval_model(naive_rnn, val_loader)
# print('Task1: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f}'.format(precision1, recall1, f11, roc_auc1))

precision1, recall1, f11, roc_auc1, precision2, recall2, f12, roc_auc2, n, p1, p2 = eval_model(naive_rnn, val_loader)
print('Task1: N={} Prevalence={:.3f} P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f}'.format(n, p1/n, precision1, recall1, f11, roc_auc1))
print('Task2: N={} Prevalence={:.3f} P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f}'.format(n, p2/n, precision2, recall2, f12, roc_auc2))

Task1: N=1885 Prevalence=0.279 P=0.363 R=0.352 F1=0.357 ROC AUC=0.582
Task2: N=1885 Prevalence=0.051 P=0.074 R=0.042 F1=0.053 ROC AUC=0.589


In [25]:
def train(model, train_loader, val_loader, combined_loader, n_epochs):
    model.train()
    for epoch in range(n_epochs):
        train_loss = 0
        for x, x_other, masks, y in train_loader:
            optimizer.zero_grad()
            y_pred = model.forward(x, x_other, masks)
            loss = criterion(torch.flatten(y_pred), torch.flatten(y))
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()
        val_loss = 0
        for x, x_other, masks, y in val_loader:
            optimizer.zero_grad()
            y_pred = model.forward(x, x_other, masks)
            loss = criterion(torch.flatten(y_pred), torch.flatten(y))
            loss.backward()
            val_loss+=loss.item()
        print('Epoch {}: training loss = {}  validation loss = {}'.format(epoch, train_loss, val_loss))
        precision1, recall1, f11, roc_auc1, precision2, recall2, f12, roc_auc2, n, p1, p2 = eval_model(model, val_loader)
        print('  Validation Set N={}\n\tTask1: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f} Prevalence={:.3f} \n\tTask2: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f} Prevalence={:.3f}'.format(n, precision1, recall1, f11, roc_auc1, p1/n, precision2, recall2, f12, roc_auc2, p2/n))
        precision1, recall1, f11, roc_auc1, precision2, recall2, f12, roc_auc2, n, p1, p2 = eval_model(model, train_loader)
        print('  Training Set N={}\n\tTask1: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f} Prevalence={:.3f} \n\tTask2: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f} Prevalence={:.3f}'.format(n, precision1, recall1, f11, roc_auc1, p1/n, precision2, recall2, f12, roc_auc2, p2/n))
        # precision1, recall1, f11, roc_auc1, precision2, recall2, f12, roc_auc2, n, p1, p2 = eval_model(model, combined_loader)
        # print('  Combined Set N={}\n\tTask1: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f} Prevalence={:.3f} \n\tTask2: P={:.3f} R={:.3f} F1={:.3f} ROC AUC={:.3f} Prevalence={:.3f}'.format(n, precision1, recall1, f11, roc_auc1, p1/n, precision2, recall2, f12, roc_auc2, p2/n))
        
# number of epochs to train the model
n_epochs = 20
train(naive_rnn, train_loader, val_loader, combined_loader, n_epochs)

Epoch 0: training loss = 109.88243162631989  validation loss = 27.95983412861824
  Validation Set N=1885
	Task1: P=0.000 R=0.000 F1=0.000 ROC AUC=0.529 Prevalence=0.279 
	Task2: P=0.000 R=0.000 F1=0.000 ROC AUC=0.451 Prevalence=0.051


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMe

  Training Set N=5652
	Task1: P=0.000 R=0.000 F1=0.000 ROC AUC=0.524 Prevalence=0.274 
	Task2: P=0.000 R=0.000 F1=0.000 ROC AUC=0.481 Prevalence=0.057
Epoch 1: training loss = 74.50674030184746  validation loss = 23.974238634109497


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  Validation Set N=1885
	Task1: P=0.000 R=0.000 F1=0.000 ROC AUC=0.544 Prevalence=0.279 
	Task2: P=0.000 R=0.000 F1=0.000 ROC AUC=0.458 Prevalence=0.051


...

  Training Set N=5652
	Task1: P=0.794 R=0.189 F1=0.306 ROC AUC=0.724 Prevalence=0.274 
	Task2: P=0.000 R=0.000 F1=0.000 ROC AUC=0.812 Prevalence=0.057
Epoch 7: training loss = 61.59763365983963  validation loss = 23.33986420929432
  Validation Set N=1885
	Task1: P=0.571 R=0.152 F1=0.240 ROC AUC=0.590 Prevalence=0.279 
	Task2: P=0.000 R=0.000 F1=0.000 ROC AUC=0.580 Prevalence=0.051


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  Training Set N=5652
	Task1: P=0.823 R=0.228 F1=0.357 ROC AUC=0.763 Prevalence=0.274 
	Task2: P=0.000 R=0.000 F1=0.000 ROC AUC=0.847 Prevalence=0.057
Epoch 8: training loss = 58.53101445734501  validation loss = 23.841863438487053


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  Validation Set N=1885
	Task1: P=0.500 R=0.202 F1=0.287 ROC AUC=0.590 Prevalence=0.279 
	Task2: P=0.000 R=0.000 F1=0.000 ROC AUC=0.590 Prevalence=0.051
  Training Set N=5652
	Task1: P=0.799 R=0.331 F1=0.468 ROC AUC=0.795 Prevalence=0.274 
	Task2: P=0.500 R=0.003 F1=0.006 ROC AUC=0.874 Prevalence=0.057
Epoch 9: training loss = 55.1456031948328  validation loss = 24.454258769750595
  Validation Set N=1885
	Task1: P=0.490 R=0.192 F1=0.276 ROC AUC=0.588 Prevalence=0.279 
	Task2: P=0.000 R=0.000 F1=0.000 ROC AUC=0.586 Prevalence=0.051
  Training Set N=5652
	Task1: P=0.859 R=0.343 F1=0.490 ROC AUC=0.831 Prevalence=0.274 
	Task2: P=0.750 R=0.028 F1=0.054 ROC AUC=0.897 Prevalence=0.057
Epoch 10: training loss = 51.48483915627003  validation loss = 25.143074110150337
  Validation Set N=1885
	Task1: P=0.461 R=0.224 F1=0.302 ROC AUC=0.590 Prevalence=0.279 
	Task2: P=0.000 R=0.000 F1=0.000 ROC AUC=0.586 Prevalence=0.051
  Training Set N=5652
	Task1: P=0.866 R=0.447 F1=0.590 ROC AUC=0.861 Prevalen